In [1]:
import sagemaker
from sagemaker import get_execution_role
role = get_execution_role()
sagemaker_session = sagemaker.Session()

In [2]:
!touch train.py
from sagemaker.tensorflow.serving import Model
sagemaker_model = Model(model_data = 's3://' + sagemaker_session.default_bucket() + '/model/model.tar.gz',
                                  role = role,
                                  entry_point = 'train.py')

In [3]:
%%time
predictor = sagemaker_model.deploy(initial_instance_count=1,
                                   instance_type='ml.m4.xlarge')

-------------!CPU times: user 709 ms, sys: 46.1 ms, total: 755 ms
Wall time: 6min 33s


In [4]:
predictor.endpoint

'sagemaker-tensorflow-serving-2020-05-15-19-36-07-163'

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adadelta
import pandas as pd
import io
import os
import requests
import numpy as np
from sklearn import metrics
from tensorflow.keras import backend as K

img_rows, img_cols = 28, 28
num_classes = 10
batch_size = 128
epochs = 1
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
    
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [44]:
data = x_test[0]
data = data.reshape(1,28,28,1)

In [45]:
data.shape

(1, 28, 28, 1)

In [46]:
import json
inp = {'instances': data}
predictor.predict(inp)

{'predictions': [[1.13809e-06,
   1.52869e-07,
   5.33329e-06,
   7.96886e-06,
   1.51839e-08,
   8.32324e-09,
   3.80715e-10,
   0.999984,
   1.2186e-07,
   1.31789e-06]]}